In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
movements_folder_path = "movements"
movements = os.listdir(movements_folder_path)
movements.remove(".DS_Store")

min_rows_number = float("inf")

for movement in movements:
    file_list = os.listdir(os.path.join(movements_folder_path, movement))
    print(f"Calculate movement {movement}")

    for file in file_list:
        file_path = os.path.join(movements_folder_path, movement, file)
        df = pd.read_csv(file_path)
        min_rows_number = min(df.shape[0], min_rows_number)

    print(f"Min rows after {movement} equals {min_rows_number}")

print(f"==========================================\nMinimum of rows in files equal {min_rows_number}")

Calculate movement 1-down
Min rows after 1-down equals 4155
Calculate movement 2-up
Min rows after 2-up equals 4155
Calculate movement 0-static
Min rows after 0-static equals 4155
Minimum of rows in files equal 4155


In [3]:
for movement in movements:
    file_list = os.listdir(os.path.join(movements_folder_path, movement))
    print(f"Resize movement {movement}")

    for file in file_list:
        file_path = os.path.join(movements_folder_path, movement, file)
        df = pd.read_csv(file_path)
        
        state = 0
        while df.shape[0] != min_rows_number:
            if state == 0:
                df = df.drop(df.index[0])
                state = 1
            else:
                df = df.drop(df.index[-1])
                state = 0

        df.to_csv(file_path, index=False)

Resize movement 1-down
Resize movement 2-up
Resize movement 0-static


In [4]:
features = []

for movement in movements:
    file_list = os.listdir(os.path.join(movements_folder_path, movement))

    for file in file_list:
        file_path = os.path.join(movements_folder_path, movement, file)
        df = pd.read_csv(file_path)
        features.append(df)

print(f"length = {len(features)}")

for feature in features:
    if feature.shape[0] != min_rows_number:
        print("not same size")
        break

print("same size")

length = 303
same size


Теперь попробуем обучить модель для классификации 3-х движений:
* **0** (статика)
* **1** (движение кисти вниз)
* **2** (движение кисти вверх)

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [6]:
my_dataset = pd.DataFrame(columns=["data", "target"])
cols = [
    "Time (sec)",
    "Pitch (deg)",
    "Roll (deg)",
    "Yaw(deg)",
    "X-Acceleration (g)",
    "Y-Acceleration (g)",
    "Z-Acceleration (g)",
    "Pitch Rate (deg/sec)",
    "Roll Rate (deg/sec)",
    "Yaw Rate (deg/sec)"
]

In [7]:
def matrix_to_vector(matrix):
    res = []

    for row in matrix:
        for element in row:
            res.append(element)
    
    return res

In [8]:
test_matrix = [
    [1, 2, 3, 4, 5],
    [6, 1, 2, 3, 4],
    [1, 3, 1, 2, 3],
    [1, 1, 1, 1, 1],
    [1, 2, 2, 2, 2]
]

matrix_to_vector(test_matrix)

[1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 1, 3, 1, 2, 3, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2]

In [50]:
for movement in movements:
    file_list = os.listdir(os.path.join(movements_folder_path, movement))

    for file in file_list:
        file_path = os.path.join(movements_folder_path, movement, file)

        temp = pd.read_csv(file_path)
        temp = temp[cols]
        matrix = temp.to_numpy()
        vector = matrix_to_vector(matrix)

        if movement == "0-static":
            my_dataset.loc[len(my_dataset.index)] = [vector, 0]
        elif movement == "1-down":
            my_dataset.loc[len(my_dataset.index)] = [vector, 1]
        elif movement == "2-up":
            my_dataset.loc[len(my_dataset.index)] = [vector, 2]

In [63]:
my_dataset['data']

0      [729705840.312976, -1.531345, 3.288962, 73.756...
1      [729717183.336099, -0.793636, 3.283734, 143.26...
2      [729707040.25009, -18.376733, 2.053893, 115.15...
3      [729705462.529353, -0.223319, 0.623548, 72.340...
4      [729706791.925182, -0.227171, 1.687749, -22.98...
                             ...                        
904    [729606399.147986, 0.700881, 0.952782, 50.2873...
905    [729600931.425837, 0.988402, 0.870238, -3.8260...
906    [729608352.74702, 1.217515, -0.662269, 55.3426...
907    [729601272.717378, 0.654508, -1.17449, -20.611...
908    [729608486.813546, 1.70852, -4.029911, 55.9137...
Name: data, Length: 909, dtype: object

In [52]:
type(my_dataset['data'][0])

list

In [53]:
len(my_dataset['data'][0])

41550

In [70]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [71]:
data = np.array(my_dataset['data'].to_list())
target = np.array(my_dataset['target'])

X_train, X_test, y_train, y_test = train_test_split(
    data,
    target,
    test_size=0.2,
    random_state=42,
    shuffle=True)


parameters = {
    'hidden_layer_sizes': [(50, 50), (100, 100), (100, 50, 25)],
    'activation': ['logistic', 'tanh', 'relu'],
    'max_iter': [1000, 1500, 2000]
}

grid_search = GridSearchCV(estimator=MLPClassifier(), param_grid=parameters, cv=3)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

model = MLPClassifier(**best_params)
model.fit(X_train, y_train)

# Предсказание целевых значений на тестовом наборе
y_pred = model.predict(X_test)
print("===============================")
print(y_pred)

# Оценка модели
accuracy = np.mean(y_pred == y_test)
print(f"Точность модели: {accuracy}")

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Точность модели: 0.26373626373626374


In [27]:
X_train, X_test, y_train, y_test = train_test_split(
    my_dataset["data"],
    my_dataset["target"],
    shuffle=True,
    test_size=0.2,
    random_state=42)

In [21]:
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

ValueError: setting an array element with a sequence.